## Deep Learning Prototype

## Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Load Training CSV File

Displays dataframe for verification purposes.

In [ ]:
# Load the training CSV file
csv_file = '../../image_labels/airspeed_labels.csv'
training_df = pd.read_csv(csv_file)

image_directory = '.../../media/airspeed_frames'
training_df['image_path'] = training_df['image_path'].apply(lambda x: os.path.join(image_directory, x))

training_df


## Helper Function - Load Image

In [ ]:
def load_image(file_path):
    image = cv2.imread(file_path)

    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

## Load Images and apply labels from CSV

In [ ]:
# Load images into a NumPy array
images = np.array([load_image(file_path) for file_path in training_df['image_path']])


# Encode the labels (assuming 'vertical_speed' is the column containing labels)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(training_df['airspeed'])

# Optionally, one-hot encode the labels (if using categorical cross-entropy)
num_classes = len(label_encoder.classes_)
one_hot_labels = tf.keras.utils.to_categorical(encoded_labels, num_classes)


# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, one_hot_labels, test_size=0.2, random_state=42)

## Epoch Notes

Best for airspeed - Epoch 13

## Execute - Run Training Model

In [ ]:
# Define the model architecture
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(110, 120, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Adjust num_classes as needed
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use appropriate loss function
              metrics=['accuracy'])

# Load and preprocess your dataset
# You can use TensorFlow's `ImageDataGenerator` for data augmentation and preprocessing.

# Train the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

# Save the trained model
model.save('airspeed_model.h5')
